# How to match? 

## 0. Import libraries 

In [1]:
# !pip install faiss-cpu
# !pip install lightautoml

In [1]:
!python --version

Python 3.8.16


In [2]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.matcher import Matcher
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [3]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,11,1,482,True
1,0,11,2,525,True
2,0,11,3,488,True
3,0,11,4,427,True
4,0,11,5,437,True


In [4]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,7,0,False,482.0,421.777778
1,10,0,False,495.0,419.777778
2,11,0,False,486.0,412.333333
3,13,0,False,507.0,413.333333
4,16,3,True,479.5,507.666667
...,...,...,...,...,...
5550,9985,0,False,508.5,436.222222
5551,9987,0,False,475.0,421.888889
5552,9990,0,False,469.5,429.555556
5553,9994,0,False,508.0,424.111111


In [5]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,7,0,0,482.0,421.777778,52,1,2
1,10,0,0,495.0,419.777778,48,0,2
2,11,0,0,486.0,412.333333,26,0,2
3,13,0,0,507.0,413.333333,32,1,1
4,16,3,1,479.5,507.666667,41,1,1


In [6]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [7]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [8]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col, n_neighbors=10)
results, quality_results = model.estimate()

[20.06.2023 14:35:37 | digitaltwins | INFO]: Applying matching


Finding index
Done
Finding index
Done


[20.06.2023 14:35:37 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 470
[20.06.2023 14:35:37 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5085
[20.06.2023 14:35:37 | Faiss digitaltwins | INFO]: Estimating quality of matching
[20.06.2023 14:35:37 | metrics | INFO]: Standardised mean difference: age           0.002108
is_male       0.000000
pre_spends    0.002732
dtype: float64
[20.06.2023 14:35:37 | metrics | INFO]: Standardised mean difference: age           0.000711
is_male       0.000000
pre_spends    0.004004
dtype: float64
[20.06.2023 14:35:37 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.9999999999999999, 'is_male': 1.0, 'pre_spends': 0.9999999995270387}
[20.06.2023 14:35:37 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.14179225415188376, 'is_male': 1.0, 'pre_spends': 0.2901663292175348}
[20.06.2023 14:35:37

In [9]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,99.773771,0.558758,0.0,98.678605,100.868936
ATC,99.759920,0.569823,0.0,98.643067,100.876773
ATT,99.923628,0.680077,0.0,98.590676,101.256580


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [10]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [11]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

[20.06.2023 14:35:37 | digitaltwins | INFO]: Applying matching


Finding index
Done
Finding index
Done
Finding index
Done
Finding index
Done


[20.06.2023 14:35:38 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 470
[20.06.2023 14:35:38 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5085
[20.06.2023 14:35:38 | Faiss digitaltwins | INFO]: Estimating quality of matching
[20.06.2023 14:35:38 | metrics | INFO]: Standardised mean difference: age           0.083486
is_male       0.082272
pre_spends    0.107855
dtype: float64
[20.06.2023 14:35:38 | metrics | INFO]: Standardised mean difference: age           0.092662
is_male       0.013177
pre_spends    0.023535
dtype: float64
[20.06.2023 14:35:38 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.022117461071978834, 'is_male': 0.0926091053587308, 'pre_spends': 0.5197453752275736}
[20.06.2023 14:35:38 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 2.6374269491295828e-05, 'is_male': 0.9933679055978988, 'pre_spends': 0.072

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [13]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [14]:
features_importance = model.lama_feature_select()
features_importance

[20.06.2023 14:35:45 | digitaltwins | INFO]: Counting feature importance
[20.06.2023 14:35:45 | lama_feature_selector | INFO]: Getting feature scores


,Feature,Importance
0,pre_spends,731901.420776
3,industry,731901.420776
1,age,411871.519897
2,is_male,47985.460449


In [15]:
features = features_importance['Feature'].to_list()

In [16]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

[20.06.2023 14:35:53 | digitaltwins | INFO]: Applying matching


Finding index
Done
Finding index
Done
Finding index
Done
Finding index
Done


[20.06.2023 14:35:53 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 470
[20.06.2023 14:35:53 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5085
[20.06.2023 14:35:53 | Faiss digitaltwins | INFO]: Estimating quality of matching
[20.06.2023 14:35:53 | metrics | INFO]: Standardised mean difference: age           0.094233
is_male       0.069506
pre_spends    0.049893
dtype: float64
[20.06.2023 14:35:53 | metrics | INFO]: Standardised mean difference: age           0.078014
is_male       0.012783
pre_spends    0.034165
dtype: float64
[20.06.2023 14:35:53 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.032463286535320185, 'is_male': 0.009750918612551032, 'pre_spends': 0.5728752383968168}
[20.06.2023 14:35:53 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.001649770877043844, 'is_male': 0.9933679055978988, 'pre_spends': 0.026

In [17]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,user_id,signup_month,pre_spends_matched,age_matched,is_male_matched,user_id_matched,signup_month_matched,industry_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,4400,498.0,69,1,1,7919,3,477.5,41.0,1.0,3812.0,0.0,2,[141],529.000000,414.000000,115.000000,1,0
1,2251,509.0,20,0,1,4055,3,493.0,67.0,0.0,6813.0,0.0,2,[2814],519.666667,432.444444,87.237613,1,0
2,2222,474.0,50,0,1,3999,3,497.5,52.0,0.0,70.0,0.0,2,[4311],519.555556,443.777778,75.777778,1,0
3,2226,470.0,30,0,1,4005,3,455.0,59.0,1.0,371.0,0.0,1,[1754],511.555556,410.444444,101.128093,1,0
4,2230,487.5,54,1,1,4011,3,495.5,41.0,1.0,3976.0,0.0,1,[5110],522.222222,420.555556,101.666667,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,2467,479.0,21,0,2,4427,0,482.5,47.0,0.0,1164.0,0.0,1,[2049],419.888889,522.111111,101.816672,0,1
5081,2468,485.0,64,0,2,4430,0,490.0,39.0,0.0,5480.0,0.0,1,[1407],424.000000,520.444444,96.451564,0,1
5082,2469,477.5,53,0,2,4431,0,504.5,44.0,1.0,6124.0,0.0,1,[5053],424.555556,510.111111,85.572399,0,1
5083,2447,486.5,52,0,2,4399,0,474.0,40.0,0.0,9542.0,0.0,1,[1824],431.333333,522.444444,90.685157,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [18]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,99.741828,0.550918,0.0,98.662029,100.821628
ATC,99.787987,0.561974,0.0,98.686519,100.889455
ATT,99.242432,0.658704,0.0,97.951372,100.533493


### 3.2 SMD, PSI, KS-test, repeats

In [19]:
# matching quality result - SMD
model.quality_result

{'psi':                 column  anomaly_score check_result                 column  \
 0          age_treated           0.07           OK          age_untreated   
 1     industry_treated           0.00           OK     industry_untreated   
 2  post_spends_treated          16.11          NOK  post_spends_untreated   
 3   pre_spends_treated           0.03           OK   pre_spends_untreated   
 
    anomaly_score check_result  
 0           0.02           OK  
 1           0.00           OK  
 2           8.27          NOK  
 3           0.01           OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       0.032463                0.001650
 is_male                   0.009751                0.993368
 pre_spends                0.572875                0.026927,
 'smd':             match_control_to_treat  match_treat_to_control
 age                       0.094233                0.078014
 is_male                   0.069506                0.012

In [20]:
# matching quality result - PSI
model.quality_result['psi']

,column,anomaly_score,check_result,column,anomaly_score,check_result
0,age_treated,0.07,OK,age_untreated,0.02,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.27,NOK
3,pre_spends_treated,0.03,OK,pre_spends_untreated,0.01,OK


In [21]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,0.032463,0.001650
is_male,0.009751,0.993368
pre_spends,0.572875,0.026927


In [22]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.95, 'match_treat_to_control': 0.09}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [23]:
 
model.validate_result()
 

[20.06.2023 14:36:19 | digitaltwins | INFO]: Applying validation of result


Finding index
Done
Finding index
Done
Finding index
Done
Finding index
Done


[20.06.2023 14:36:19 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 470
[20.06.2023 14:36:19 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5085


LinAlgError: Matrix is not positive definite

In [ ]:
model.matcher.df_matched